# IA Générative

## 1. Génération de Recommandations Musicales

Cette section transforme une intention utilisateur en une fiche technique musicale structurée. Le processus se déroule en trois étapes clés :

Extraction Sémantique : Nous utilisons un modèle de langage (Mistral AI via LangChain) pour analyser la requête naturelle de l'utilisateur. L'objectif est d'isoler des variables précises comme le genre souhaité, le tempo (BPM) ou la tonalité.

Analyse Statistique : À partir de ces variables, le code interroge un jeu de données musical (dataset.csv). Il filtre les morceaux les plus populaires correspondant aux critères pour calculer des « paramètres optimaux » (énergie médiane, valence pour l'humeur, etc.). Cela permet de s'appuyer sur des standards musicaux qui fonctionnent réellement.

Conseils Artistiques : Enfin, l'IA synthétise ces données pour fournir trois conseils de composition concrets (rythme, accords et ambiance), offrant ainsi une base créative solide à l'utilisateur.

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_mistralai.chat_models import ChatMistralAI
from typing import Optional
from pydantic import BaseModel

# Chargement des variables d'environnement (clé API Mistral située dans le fichier .env)
load_dotenv()

# Initialisation du modèle de langage Mistral AI
llm = ChatMistralAI(model="mistral-small-latest", mistral_api_key=os.getenv("MISTRAL_API_KEY"))

# Définition d'un schéma de données (Pydantic) pour forcer l'IA à extraire des champs précis
# Cela garantit que la réponse de l'IA sera toujours un objet structuré et non du texte libre.
class MusicParams(BaseModel):
    genre: Optional[str] = None
    tempo: Optional[int] = None
    key: Optional[str] = None
    duration: Optional[float] = None

def extract_parameters(user_input):
    """
    Analyse la requête utilisateur pour en extraire les métadonnées musicales.
    Utilise la fonction 'with_structured_output' de LangChain pour obtenir un dictionnaire propre.
    """
    try:
        # On envoie la requête à l'IA en lui imposant le schéma MusicParams
        return llm.with_structured_output(MusicParams).invoke(f"Extrait en JSON : {user_input}").model_dump()
    except:
        # En cas d'erreur (ex: API injoignable), on renvoie une structure vide par défaut
        return {k: None for k in ["genre", "tempo", "key", "duration"]}


def get_params(genre=None, tempo=None):
    """
    Analyse le dataset CSV pour trouver les réglages idéaux basés sur les morceaux populaires.
    """
    # Chargement de la base de données musicale
    df = pd.read_csv('../archive/dataset.csv')
    
    # On filtre uniquement les morceaux situés dans le top 20% de popularité (références à succès)
    top = df[df.popularity >= df.popularity.quantile(0.8)]
    
    # Filtrage par genre : si le genre demandé existe avec assez de données, on s'y restreint
    f = top[top.track_genre == genre] if genre and len(top[top.track_genre == genre]) > 10 else top
    
    # Filtrage par tempo : on cherche des morceaux à +/- 10 BPM de la demande
    if tempo:
        t_f = f[(f.tempo >= tempo-10) & (f.tempo <= tempo+10)]
        # Si le filtre est trop précis (moins de 5 titres), on garde le groupe précédent pour plus de fiabilité
        f = t_f if len(t_f) >= 5 else top
        
    # Calcul de la médiane de chaque caractéristique pour définir le profil musical "type"
    res = f[['danceability', 'energy', 'key', 'mode', 'valence', 'tempo', 'duration_ms']].median().to_dict()
    
    # On force le tempo précis demandé par l'utilisateur s'il existe
    if tempo: res['tempo'] = tempo
    return res, len(f)


# Récupération de l'intention utilisateur via la console
user_input = input("Quelle musique veux-tu créer ? ")

# Étape 1 : Extraction intelligente des paramètres via le LLM
p_in = extract_parameters(user_input)

# Étape 2 : Recherche statistique des valeurs optimales dans le dataset
p_opt, n = get_params(p_in['genre'], p_in['tempo'])

# Correspondance des index musicaux (0=Do, 1=Do#, etc.) pour un affichage lisible
keys_fr = ["Do", "Do#", "Ré", "Ré#", "Mi", "Fa", "Fa#", "Sol", "Sol#", "La", "La#", "Si"]

# Étape 3 : Préparation d'un dictionnaire formaté pour l'affichage final
fmt = {
    "Genre": p_in['genre'].capitalize() if p_in['genre'] else "Standard",
    "Tonalité": f"{keys_fr[int(p_opt['key'])]} {'Maj' if p_opt['mode'] > 0.5 else 'Min'}",
    "Tempo": f"{p_opt['tempo']:.0f} BPM",
    "Énergie": f"{p_opt['energy']:.0%}", # Formate le float (ex: 0.65) en pourcentage (65%)
    "Humeur": f"{p_opt['valence']:.0%}"
}

# Étape 4 : Affichage des recommandations
print(f"\nRecommandations (basées sur {n} titres similaires) :", *[f"\n - {k}: {v}" for k, v in fmt.items()])

# Étape 5 : Exécution
prompt = f"Donne 3 conseils courts (Rythme, Accords, Ambiance) pour un morceau : {fmt['Genre']}, {fmt['Tempo']}, {fmt['Tonalité']}."
print(f"\nConseils de l'IA :\n{llm.invoke(prompt).content}")

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

## 2. Génération d'une Musique

Cette seconde partie du projet convertit les recommandations textuelles et techniques en un véritable signal audio.

Nous utilisons le modèle MusicGen de Meta (via la bibliothèque Transformers d'Hugging Face). Il s'agit d'un modèle de Deep Learning de type Transformeur capable de générer de la musique de manière conditionnelle. Le code injecte des paramètres précédemment définis (style, ambiance et tempo) pour créer une description textuelle que le modèle interprète pour composer une mélodie originale sous forme de fichier audio.

Pour installer la bibliothèque Transformers : \
pip install git+https://github.com/huggingface/transformers.git scipy torch

In [ ]:
import numpy as np
from transformers import MusicgenForConditionalGeneration, AutoProcessor
from IPython.display import Audio

def generate_music_direct(p_in, p_opt):
    """
    Utilise le modèle MusicGen pour transformer les recommandations en signal audio.
    Prend en entrée les intentions utilisateur (p_in) et les statistiques du dataset (p_opt).
    """
    # Sélection du modèle "small" de Meta pour une génération rapide sur CPU/GPU
    model_id = "facebook/musicgen-small"
    
    # Chargement du processeur de texte et du modèle de synthèse sonore
    processor = AutoProcessor.from_pretrained(model_id)
    model = MusicgenForConditionalGeneration.from_pretrained(model_id)

    # Construction dynamique du prompt textuel basé sur les résultats de l'étape 1
    # On combine le genre, l'humeur calculée (valence) et le tempo idéal
    genre = p_in['genre'] if p_in['genre'] else 'pop'
    description = f"{genre} music, mood: {p_opt['valence']*100:.0f}% happy, {p_opt['tempo']:.0f} BPM"

    # Encodage de la description pour le modèle
    inputs = processor(
        text=[description],
        padding=True,
        return_tensors="pt",
    )

    print(f"Génération lancée : {description}")
    
    # Génération des échantillons audio
    # max_new_tokens=256 génère environ 5 secondes de musique
    audio_values = model.generate(**inputs, max_new_tokens=256)

    # Récupération de la fréquence d'échantillonnage native du modèle (ex: 32kHz)
    sampling_rate = model.config.audio_encoder.sampling_rate
    
    # Conversion du tenseur brut en tableau NumPy compatible avec le lecteur audio
    audio_data = audio_values.squeeze().detach().cpu().numpy().astype(np.float32)

    # Affichage du widget de lecture si la donnée est valide
    if audio_data.size > 1:
        print("Génération terminée !")
        display(Audio(audio_data, rate=sampling_rate))
    else:
        print("ÉCHEC DE LA GÉNÉRATION")
    
    return audio_data

# Appel de la fonction
audio_result = generate_music_direct(p_in, p_opt)

Loading weights:   0%|          | 0/611 [00:00<?, ?it/s]

MusicgenForConditionalGeneration LOAD REPORT from: facebook/musicgen-small
Key                                           | Status     |  | 
----------------------------------------------+------------+--+-
decoder.model.decoder.embed_positions.weights | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Génération lancée : Rock music, mood: 49% happy, 100 BPM
Génération terminée !


### Remarques :

La première exécution prend un certain temps car on charge le modèle localement.

La génération ayant besoin d'une quantité importante de ressources, nous limitons la durée (ou le nombre de tokens) afin de ne pas rendre la génération trop lente.

La durée de la musique générée n'est donc pas égale à la durée recommandée par l'IA, mais cela aurait été possible dans un temps raisonnable si nous avions accès à une machine puissante dédiée par exemple.